In [ ]:
import pandas as pd
import datetime
import numpy as np
import re
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [ ]:
#To display all the columns and rows
pd.set_option('display.max_columns', 1000);
pd.set_option('display.max_rows', 1000);
pd.set_option('display.width', 1000);
#To ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#To supress scientific notations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Loading all the required files

In [ ]:
%%time
#File containing extracted data from orphanet and mimic
merged_admissions = pd.read_excel("all_disorders_details.xlsx")
merged_adm_pat_sorted = pd.read_excel("merged_adm_pat_sorted.xlsx")

CPU times: user 4min 2s, sys: 878 ms, total: 4min 3s
Wall time: 4min 3s


In [ ]:
%%time
#Smaller files from MIMIC IV
icu_stays = pd.read_csv("mimic-iv-2.1/icu/icustays.csv")
d_icd_diagnoses = pd.read_csv("mimic-iv-2.1/hosp/d_icd_diagnoses.csv")
diagnoses_icd = pd.read_csv("mimic-iv-2.1/hosp/diagnoses_icd.csv")
d_icd_procedures = pd.read_csv("mimic-iv-2.1/hosp/d_icd_procedures.csv")
procedures_icd = pd.read_csv("mimic-iv-2.1/hosp/procedures_icd.csv")
labitems = pd.read_csv("mimic-iv-2.1/hosp/d_labitems.csv")
transfers = pd.read_csv("mimic-iv-2.1/hosp/transfers.csv")
patients = pd.read_csv("mimic-iv-2.1/hosp/patients.csv")
admissions = pd.read_csv("mimic-iv-2.1/hosp/admissions.csv")
omr = pd.read_csv("mimic-iv-2.1/hosp/omr.csv")
microbiologyevents = pd.read_csv("mimic-iv-2.1/hosp/microbiologyevents.csv")
#Larger files from MIMIC IV
pharmacy = pd.read_csv("mimic-iv-2.1/hosp/pharmacy.csv")
poe = pd.read_csv("mimic-iv-2.1/hosp/poe.csv")
prescriptions = pd.read_csv("mimic-iv-2.1/hosp/prescriptions.csv")
#Largest file from MIMIC IV
labevents = pd.read_csv("mimic-iv-2.1/hosp/labevents.csv")

<timed exec>:12: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:14: DtypeWarning: Columns (16,18,24,26) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 7min 15s, sys: 5min 31s, total: 12min 47s
Wall time: 14min 59s


In [ ]:
#Checking counts
merged_admissions['class'].value_counts()

4    30896
3    21546
5     6715
2     2207
1     1375
Name: class, dtype: int64

## Extracting features and combining to the main table

#### 1. Admissions Table

In [ ]:
#Ensure datetime columns are in datetime format
merged_admissions['admittime'] = pd.to_datetime(merged_admissions['admittime'], errors='coerce')
merged_admissions['dischtime'] = pd.to_datetime(merged_admissions['dischtime'], errors='coerce')
merged_admissions['edregtime'] = pd.to_datetime(merged_admissions['edregtime'], errors='coerce')
merged_admissions['dod'] = pd.to_datetime(merged_admissions['dod'], errors='coerce')

# 1. Alive or Dead
merged_admissions['is_dead'] = merged_admissions['dod'].notnull().astype(int)

# 2. Length of Stay (LOS) in days
merged_admissions['los'] = (merged_admissions['dischtime'] - merged_admissions['admittime']).dt.total_seconds() / (24 * 60 * 60)

# 3. Admission Season
def get_season(date):
    if pd.isnull(date):
        return "Unknown"
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

merged_admissions['admission_season'] = merged_admissions['admittime'].apply(get_season)

# 4. Time to Emergency Transfer (in hours)
merged_admissions['time_to_emergency_hrs'] = (merged_admissions['edregtime'] - merged_admissions['admittime']).dt.total_seconds() / 3600

# 5. Admit Month (1 to 12)
merged_admissions['admit_month'] = merged_admissions['admittime'].dt.month

# 6. Nighttime Admission (1 if between 8 PM and 6 AM)
merged_admissions['admit_hour'] = merged_admissions['admittime'].dt.hour
merged_admissions['is_night_admit'] = merged_admissions['admit_hour'].apply(lambda h: 1 if h >= 20 or h < 6 else 0)

# 7. Weekend Admission (1 if Saturday or Sunday)
merged_admissions['admit_weekday'] = merged_admissions['admittime'].dt.weekday
merged_admissions['is_weekend_admit'] = merged_admissions['admit_weekday'].apply(lambda x: 1 if x >= 5 else 0)

#Drop intermediate columns
merged_admissions.drop(columns=['admit_hour', 'admit_weekday'], inplace=True)

In [ ]:
merged_admissions.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62739 entries, 0 to 62738
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   icd_code               62739 non-null  object        
 1   long_title             62739 non-null  object        
 2   subject_id             62739 non-null  int64         
 3   hadm_id                62739 non-null  int64         
 4   seq_num                62739 non-null  int64         
 5   Classification         62739 non-null  object        
 6   included               62739 non-null  object        
 7   class                  62739 non-null  int64         
 8   gender                 62739 non-null  object        
 9   anchor_age             62739 non-null  int64         
 10  anchor_year            62739 non-null  int64         
 11  anchor_year_group      62739 non-null  object        
 12  dod                    10757 non-null  datetime64[ns]
 13  a

In [ ]:
merged_admissions.head()

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit
0,E785,"Hyperlipidemia, unspecified",10000084,23052089,5,common_diseases,True,4,M,72,2160,2017 - 2019,2161-02-13,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,1,1,0,E785,1,4.539,Fall,-5.333,11,1,0
1,K219,Gastro-esophageal reflux disease without esoph...,10000117,22927623,4,common_diseases,True,4,F,48,2174,2008 - 2010,NaT,2181-11-15 02:05:00,2181-11-15 14:52:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,DIVORCED,WHITE,2181-11-14 21:51:00,2181-11-15 09:57:00,0,1,1,0,"K219,F419,I341",0,0.533,Fall,-4.233,11,1,0
2,K219,Gastro-esophageal reflux disease without esoph...,10000980,25911675,22,common_diseases,True,4,F,73,2186,2008 - 2010,2193-08-26,2191-05-23 15:33:00,2191-05-24 17:14:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2191-05-22 16:06:00,2191-05-23 17:56:00,0,1,2,0,K219,1,1.070,Spring,-23.450,5,0,0
3,F419,"Anxiety disorder, unspecified",10001180,27864856,9,common_diseases,True,4,F,33,2192,2014 - 2016,NaT,2197-06-06 08:12:00,2197-06-10 10:15:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,NaT,NaN,0,1,0,0,F419,0,4.085,Summer,NaN,6,0,0
4,C675,Malignant neoplasm of bladder neck,10001401,21544441,1,"rare_neoplastic_diseases, rare_urogenital_dise...",True,3,F,89,2131,2014 - 2016,NaT,2131-06-04 00:00:00,2131-06-15 16:10:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaN,0,1,5,0,"C675,E785,I10",0,11.674,Summer,NaN,6,1,0


#### 2. Procedures Table

In [ ]:
#Combining procedures_icd and d_ic_procedures based on icd_code
combinedprocedures = pd.merge(procedures_icd, d_icd_procedures, on='icd_code', how='left')

#Keeping only those records which have an entry in the main merged dataframe based on subject_id and hadm_id
filtered_procedures = combinedprocedures.merge(merged_admissions[['subject_id', 'hadm_id']].drop_duplicates(), on=['subject_id', 'hadm_id'],how='right')

#Since there are multiple procedures done on a single subject_id and hadm_id combining the multiple procedures under one
merged_procedures = (filtered_procedures.groupby(['subject_id', 'hadm_id'])['long_title']
    .apply(lambda x: '; '.join(x.unique().astype(str)))  # Convert to string before joining
    .reset_index()
    .rename(columns={'long_title': 'procedures_applied'}))

#Combining with the main merged dataframe
merged_procedures = merged_procedures.merge(merged_admissions, on=['subject_id', 'hadm_id'], how='right')

#Generating new features from procedures table

#1. Count total number of procedures per subject_id, hadm_id
procedure_counts = (filtered_procedures.groupby(['subject_id', 'hadm_id'])['long_title'].count().reset_index()
.rename(columns={'long_title': 'total_no_of_procedures'}))

#2. Count number of unique procedures per subject_id, hadm_id
unique_procedure_counts = (filtered_procedures.groupby(['subject_id', 'hadm_id'])['long_title'].nunique().reset_index()
.rename(columns={'long_title': 'no_of_unique_procedures'}))

#3. ICU/critical procedures

#Define ICU/critical procedures related keywords

icu_keywords = [
    'ventilation', 'mechanical ventilation', 'intubation', 'oxygenation', 'tracheostomy', 'monitoring', 'hemodynamic', 'icu', 'critical care', 'respiratory', 'pulmonary',
    'cardiac', 'resuscitation', 'central line', 'arterial catheter', 'venous access', 'catheter insertion', 'vasopressor', 'life support', 'airway', 'endotracheal', 'sedation',
    'pressure support', 'percutaneous', 'neuro monitoring', 'neuromuscular']

icu_mask = filtered_procedures['long_title'].fillna('').str.lower().apply(
    lambda x: any(keyword in x for keyword in icu_keywords)
)

icu_procedures = filtered_procedures[icu_mask]

icu_counts = (
    icu_procedures
    .groupby(['subject_id', 'hadm_id'])['long_title']
    .count()
    .reset_index()
    .rename(columns={'long_title': 'count_icu_critical_procedures'})
)

#Merge these counts into merged_procedures
merged_procedures = merged_procedures.merge(procedure_counts, on=['subject_id', 'hadm_id'], how='left')
merged_procedures = merged_procedures.merge(unique_procedure_counts, on=['subject_id', 'hadm_id'], how='left')
merged_procedures = merged_procedures.merge(icu_counts, on=['subject_id', 'hadm_id'], how='left')

#Fill NaNs with 0 and convert counts to int
merged_procedures['total_no_of_procedures'] = merged_procedures['total_no_of_procedures'].fillna(0).astype(int)
merged_procedures['no_of_unique_procedures'] = merged_procedures['no_of_unique_procedures'].fillna(0).astype(int)
merged_procedures['count_icu_critical_procedures'] = merged_procedures['count_icu_critical_procedures'].fillna(0).astype(int)

In [ ]:
merged_procedures.head()

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures
0,10000084,23052089,nan,E785,"Hyperlipidemia, unspecified",5,common_diseases,True,4,M,72,2160,2017 - 2019,2161-02-13,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0,1,1,0,E785,1,4.539,Fall,-5.333,11,1,0,0,0,0
1,10000117,22927623,nan,K219,Gastro-esophageal reflux disease without esoph...,4,common_diseases,True,4,F,48,2174,2008 - 2010,NaT,2181-11-15 02:05:00,2181-11-15 14:52:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,DIVORCED,WHITE,2181-11-14 21:51:00,2181-11-15 09:57:00,0,1,1,0,"K219,F419,I341",0,0.533,Fall,-4.233,11,1,0,0,0,0
2,10000980,25911675,nan,K219,Gastro-esophageal reflux disease without esoph...,22,common_diseases,True,4,F,73,2186,2008 - 2010,2193-08-26,2191-05-23 15:33:00,2191-05-24 17:14:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2191-05-22 16:06:00,2191-05-23 17:56:00,0,1,2,0,K219,1,1.070,Spring,-23.450,5,0,0,0,0,0
3,10001180,27864856,"Extraction of Products of Conception, Low, Ope...",F419,"Anxiety disorder, unspecified",9,common_diseases,True,4,F,33,2192,2014 - 2016,NaT,2197-06-06 08:12:00,2197-06-10 10:15:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,NaT,NaN,0,1,0,0,F419,0,4.085,Summer,NaN,6,0,0,1,1,0
4,10001401,21544441,"Resection of Bladder, Percutaneous Endoscopic ...",C675,Malignant neoplasm of bladder neck,1,"rare_neoplastic_diseases, rare_urogenital_dise...",True,3,F,89,2131,2014 - 2016,NaT,2131-06-04 00:00:00,2131-06-15 16:10:00,NaN,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaN,0,1,5,0,"C675,E785,I10",0,11.674,Summer,NaN,6,1,0,8,8,6


#### 3. OMR Table

For the OMR table there are multiple body measurements but only 4 of them have values for almost all patients. Those 4 values are Blood Pressure, BMI (kg/m2), Height and Weight. Out of these we are only using BMI. Weight and Height will be highly correlated so no point in using that and blood pressure also has many records missing after consideration of dataset.

In [ ]:
# Step 1: Filter and clean omr for BMI values only
omr_bmi = omr[omr['result_name'] == 'BMI (kg/m2)'].copy()
omr_bmi = omr_bmi.dropna(subset=['result_value', 'chartdate'])

# Ensure correct types
omr_bmi['chartdate'] = pd.to_datetime(omr_bmi['chartdate'], errors='coerce')
omr_bmi['result_value'] = pd.to_numeric(omr_bmi['result_value'], errors='coerce')
omr_bmi = omr_bmi.dropna(subset=['result_value'])

# Step 2: Prepare merged_procedures
merged_procedures['admittime'] = pd.to_datetime(merged_procedures['admittime'], errors='coerce')

# Step 3: Merge to get admittime into omr_bmi for only those subject_ids present in merged_procedures
omr_bmi = omr_bmi.merge(merged_procedures[['subject_id', 'admittime']],on='subject_id', how='inner')

# Step 4: Compute time difference between BMI date and admission
omr_bmi['timediff'] = (omr_bmi['chartdate'] - omr_bmi['admittime']).abs()

# Step 5: Keep only the BMI closest to admission for each subject
closest_bmi = omr_bmi.sort_values('timediff').drop_duplicates('subject_id', keep='first')

# Step 6: Merge this final closest BMI into merged_procedures
merged_bmi = merged_procedures.merge(closest_bmi[['subject_id', 'result_value', 'chartdate']],on='subject_id',
                                     how='left')

# Rename columns for clarity
merged_bmi = merged_bmi.rename(columns={'result_value': 'bmi','chartdate': 'bmi_chartdate'})


In [ ]:
merged_bmi.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  int64         
 1   hadm_id                        62739 non-null  int64         
 2   procedures_applied             62739 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

#### 4. Transfers Table

In [ ]:
transfers.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890730 entries, 0 to 1890729
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   subject_id   1890730 non-null  int64 
 1   hadm_id      1560641 non-null  Int64 
 2   transfer_id  1890730 non-null  int64 
 3   eventtype    1890730 non-null  object
 4   careunit     1459642 non-null  object
 5   intime       1890730 non-null  object
 6   outtime      1459645 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 102.8+ MB


In [ ]:
merged_bmi.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  int64         
 1   hadm_id                        62739 non-null  int64         
 2   procedures_applied             62739 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

In [ ]:
transfers['eventtype'].value_counts()

ED           625943
admit        431098
discharge    431088
transfer     402601
Name: eventtype, dtype: int64

In [ ]:
# Align data types across both DataFrames
merged_bmi['subject_id'] = merged_bmi['subject_id'].astype('Int64')
transfers['subject_id'] = transfers['subject_id'].astype('Int64')
merged_bmi['hadm_id'] = merged_bmi['hadm_id'].astype('Int64')
transfers['hadm_id'] = transfers['hadm_id'].astype('Int64')

In [ ]:
transfers[(transfers['subject_id'] == 13196122) & (transfers['hadm_id'] == 28597267)]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
603106,13196122,28597267,30313866,discharge,NaN,2126-11-16 15:05:14,NaN
603107,13196122,28597267,32266002,transfer,PACU,2126-11-11 08:15:16,2126-11-11 08:40:04
603108,13196122,28597267,33860686,transfer,Cardiology Surgery Intermediate,2126-11-12 12:42:32,2126-11-16 15:05:14
603109,13196122,28597267,34555527,transfer,Cardiac Vascular Intensive Care Unit (CVICU),2126-11-11 08:40:04,2126-11-12 12:42:32
603110,13196122,28597267,38379336,admit,Medicine/Cardiology Intermediate,2126-11-06 19:03:30,2126-11-11 08:15:16


In [ ]:
#Getting only the transfers related to our main merged dataframe patients
filtered_transfers = transfers.merge(merged_bmi[['subject_id', 'hadm_id']].drop_duplicates(),
                                     on=['subject_id', 'hadm_id'],how='right')

#Number of transfers where eventtype == "transfer"
transfer_counts = (
    filtered_transfers[filtered_transfers['eventtype'].str.lower() == 'transfer']
    .groupby(['subject_id', 'hadm_id'])
    .size()
    .reset_index(name='no_of_transfers')
)

#Number of distinct careunits seen
careunit_counts = (
    filtered_transfers
    .groupby(['subject_id', 'hadm_id'])['careunit']
    .nunique()
    .reset_index()
    .rename(columns={'careunit': 'no_of_distinct_careunits'})
)

#Number of ED transfers
ed_transfer_counts = (
    filtered_transfers[filtered_transfers['eventtype'].str.lower()== 'ed']
    .groupby(['subject_id', 'hadm_id'])
    .size()
    .reset_index(name='no_of_ed_transfers')
)

#First Careunit (eventtype == 'admit')
first_careunit = (
    filtered_transfers[filtered_transfers['eventtype'] == 'admit']
    .sort_values(by=['subject_id', 'hadm_id', 'intime'])  # ensure correct order
    .drop_duplicates(['subject_id', 'hadm_id'], keep='first')
    [['subject_id', 'hadm_id', 'careunit']]
    .rename(columns={'careunit': 'first_careunit'})
)

# Merge all 4 into merged_transfers
merged_transfers = (
    merged_bmi
    .merge(transfer_counts, on=['subject_id', 'hadm_id'], how='left')
    .merge(careunit_counts, on=['subject_id', 'hadm_id'], how='left')
    .merge(ed_transfer_counts, on=['subject_id', 'hadm_id'], how='left')
    .merge(first_careunit, on=['subject_id', 'hadm_id'], how='left')
)

# Fill missing values with 0
merged_transfers['no_of_transfers'] = merged_transfers['no_of_transfers'].fillna(0).astype(int)
merged_transfers['no_of_distinct_careunits'] = merged_transfers['no_of_distinct_careunits'].fillna(0).astype(int)
merged_transfers['no_of_ed_transfers'] = merged_transfers['no_of_ed_transfers'].fillna(0).astype(int)


In [ ]:
#Sorting by subject_id, hadm_id, and intime (to get visit order)
sorted_transfers = (filtered_transfers.sort_values(by=['subject_id', 'hadm_id', 'intime']))

# Group by subject_id and hadm_id, combine careunits in order (drop NaNs to avoid "nan" strings)
careunit_sequence = (sorted_transfers.groupby(['subject_id', 'hadm_id'])['careunit'].apply(lambda x: ';'.join(x.astype(str))).reset_index()
.rename(columns={'careunit': 'careunit_transfer_sequence'}))

#Merge with main dataframe
merged_transfers = merged_transfers.merge(careunit_sequence, on=['subject_id', 'hadm_id'], how='left')

#Getting the last valid careunit
def get_last_valid_unit(seq):
    if pd.isnull(seq):
        return pd.NA
    units = [x for x in seq.split(';') if x.strip().lower() != 'nan' and x.strip() != '']
    return units[-1] if units else pd.NA

merged_transfers['last_careunit'] = merged_transfers['careunit_transfer_sequence'].apply(get_last_valid_unit)

In [ ]:
merged_transfers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  Int64         
 1   hadm_id                        62739 non-null  Int64         
 2   procedures_applied             62739 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

#### 5. Icustays Table

In [ ]:
#Getting only the icustays related to our main merged dataframe patients
filtered_icu = icu_stays.merge(merged_transfers[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')
#Note that over 85% data is missing. So think of another way to utilize icu_stays
#Count ICU stays per subject_id and hadm_id
icu_stay_counts = (filtered_icu.dropna(subset=['stay_id']).groupby(['subject_id', 'hadm_id'])['stay_id'].count().reset_index().rename(columns={'stay_id': 'no_of_icu_stays'}))
#Merge into main dataframe
merged_icu = merged_transfers.merge(icu_stay_counts, on=['subject_id', 'hadm_id'], how='left')
#Fill NaN values with 0 (i.e., no ICU stays found)
merged_icu['no_of_icu_stays'] = merged_icu['no_of_icu_stays'].fillna(0).astype(int)

In [ ]:
merged_icu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  Int64         
 1   hadm_id                        62739 non-null  Int64         
 2   procedures_applied             62739 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

#### 6. POE Table

In [ ]:
#Filter poe to include only rows with subject_id and hadm_id in main merged table
filtered_poe = poe.merge(merged_icu[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')
#Count occurrences of each order_type per subject_id and hadm_id
order_type_counts = (filtered_poe.groupby(['subject_id', 'hadm_id', 'order_type']).size().unstack(fill_value=0).reset_index())
#Rename columns to reflect count of each order_type
order_type_counts.columns = ['subject_id', 'hadm_id'] + [f'count_{col.replace(" ", "_").lower()}' for col in order_type_counts.columns[2:]]
#Merge into merged_icu
merged_poe = merged_icu.merge(order_type_counts, on=['subject_id', 'hadm_id'], how='left')
#Fill NaNs (if any) with 0 and cast to int
order_type_cols = [col for col in merged_poe.columns if col.startswith('count_')]
merged_poe[order_type_cols] = merged_poe[order_type_cols].fillna(0).astype(int)

In [ ]:
merged_poe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 66 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  Int64         
 1   hadm_id                        62739 non-null  Int64         
 2   procedures_applied             62739 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

#### 7. Microbiologyevents Table

In [ ]:
print(merged_poe[['subject_id', 'hadm_id']].dtypes)
print(microbiologyevents[['subject_id', 'hadm_id']].dtypes)

subject_id    Int64
hadm_id       Int64
dtype: object
subject_id      int64
hadm_id       float64
dtype: object


In [ ]:
# Align data types across both DataFrames
microbiologyevents['subject_id'] = microbiologyevents['subject_id'].astype('Int64')
microbiologyevents['hadm_id'] = microbiologyevents['hadm_id'].astype('Int64')

In [ ]:
# Filter microbiologyevents to include only rows with subject_id and hadm_id in main merged table
filtered_microbiologyevents = microbiologyevents.merge(merged_poe[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')

#Filter only desired specimen types of which we want count.
desired_spec_types = ['BLOOD CULTURE', 'URINE', 'SWAB', 'STOOL', 'ABSCESS', 'TISSUE', 'SPUTUM']
filtered_micro = filtered_microbiologyevents[filtered_microbiologyevents['spec_type_desc'].isin(desired_spec_types)]

#Group and count occurrences
spec_type_counts = (filtered_micro.groupby(['subject_id', 'hadm_id', 'spec_type_desc']).size().unstack(fill_value=0).reset_index())

#Rename columns to indicate counts (lowercased and underscores)
spec_type_counts.columns = ['subject_id', 'hadm_id'] + [f'count_{col.lower().replace(" ", "_")}' for col in spec_type_counts.columns[2:]]

#Merge into main merged dataframe
merged_micro = merged_poe.merge(spec_type_counts, on=['subject_id', 'hadm_id'], how='left')

#Fill NaNs with 0 (in case of no microbiology data for some patients)
spec_cols = [col for col in merged_micro.columns if col.startswith('count_')]
merged_micro[spec_cols] = merged_micro[spec_cols].fillna(0).astype(int)

In [ ]:
#1. Total number of microbiologyevents including other tests that were missed in the main tests
total_micro = (filtered_microbiologyevents.groupby(['subject_id', 'hadm_id']).size().reset_index(name='total_microbiologyevents'))

#2. Number of positive cultures (org_name not null)
positive_cultures = (filtered_microbiologyevents[filtered_microbiologyevents['org_name'].notna()].groupby(['subject_id', 'hadm_id']).size().reset_index(name='positive_cultures'))

#3. Number of negative cultures (org_name is null)
negative_cultures = (filtered_microbiologyevents[filtered_microbiologyevents['org_name'].isna()].groupby(['subject_id', 'hadm_id']).size().reset_index(name='negative_cultures'))

#4. Number of susceptible tests (interpretation == 'S')
susceptible_tests = (filtered_microbiologyevents[filtered_microbiologyevents['interpretation'] == 'S'].groupby(['subject_id', 'hadm_id']).size()
.reset_index(name='susceptible_tests'))

#5. Number of resistant tests (interpretation == 'R')
resistant_tests = (filtered_microbiologyevents[filtered_microbiologyevents['interpretation'] == 'R'].groupby(['subject_id', 'hadm_id']).size()
.reset_index(name='resistant_tests'))

#Merge all metrics into merged_micro
from functools import reduce
#List of all aggregated DataFrames
micro_stats = [total_micro, positive_cultures, negative_cultures, susceptible_tests, resistant_tests]
# Reduce into one DataFrame
from pandas import merge
micro_summary = reduce(lambda left, right: merge(left, right, on=['subject_id', 'hadm_id'], how='outer'), micro_stats)
#Fill NaNs with 0
micro_summary = micro_summary.fillna(0).astype({
    'total_microbiologyevents': 'int',
    'positive_cultures': 'int',
    'negative_cultures': 'int',
    'susceptible_tests': 'int',
    'resistant_tests': 'int'})
# Final merge into merged_micro
merged_micro = merged_micro.merge(micro_summary, on=['subject_id', 'hadm_id'], how='left')
# Again fill NaNs if any (e.g., if some rows in merged_micro had no micro events)
cols_to_fill = ['total_microbiologyevents', 'positive_cultures', 'negative_cultures', 'susceptible_tests', 'resistant_tests']
merged_micro[cols_to_fill] = merged_micro[cols_to_fill].fillna(0).astype(int)

In [ ]:
#6. Unique organism names per subject_id and hadm_id (excluding nulls)
org_names = (filtered_microbiologyevents[filtered_microbiologyevents['org_name'].notna()].groupby(['subject_id', 'hadm_id'])['org_name'].unique()
    .apply(lambda orgs: '; '.join(sorted(set(orgs)))).reset_index(name='organisms_found'))

#Merging the organisms into main merged dataframe
merged_micro = merged_micro.merge(org_names, on=['subject_id', 'hadm_id'], how='left')

#7. First charttime per subject_id and hadm_id
first_culture_time = (filtered_microbiologyevents.groupby(['subject_id', 'hadm_id'])['charttime'].min().reset_index(name='first_culture_time'))

#Merging the first culture time into main merged dataframe
merged_micro = merged_micro.merge(first_culture_time, on=['subject_id', 'hadm_id'], how='left')

#There are some Nan in first_culture_time but that is because those patients have no entry in the microbiologyevents table.

#8. All different Spec_Types
# Convert spec_type_desc to string to handle NaNs explicitly
filtered_microbiologyevents['spec_type_desc_str'] = filtered_microbiologyevents['spec_type_desc'].astype(str)

#Group by subject_id and hadm_id, aggregate unique values, and join with semicolons
spec_types = (filtered_microbiologyevents.groupby(['subject_id', 'hadm_id'])['spec_type_desc_str'].unique().apply(lambda vals: '; '.join(sorted(set(vals))))
.reset_index(name='spec_types'))

#Merge into merged_micro
merged_micro = merged_micro.merge(spec_types, on=['subject_id', 'hadm_id'], how='left')

In [ ]:
#To correct the nan issue, below code wasn't picking it up
merged_micro = merged_micro.replace(['nan', 'NaN', 'NAN'], np.nan)

In [ ]:
#Applying some conditional fixes introduced due to table merging
def apply_conditional_fixes(df):
    # Condition: If spec_types is NaN
    mask_spec_na = df['spec_types'].isna()
    df.loc[mask_spec_na, 'total_microbiologyevents'] = 0
    df.loc[mask_spec_na, 'negative_cultures'] = 0

    return df

merged_micro = apply_conditional_fixes(merged_micro)

In [ ]:
merged_micro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 81 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     62739 non-null  Int64         
 1   hadm_id                        62739 non-null  Int64         
 2   procedures_applied             32543 non-null  object        
 3   icd_code                       62739 non-null  object        
 4   long_title                     62739 non-null  object        
 5   seq_num                        62739 non-null  int64         
 6   Classification                 62739 non-null  object        
 7   included                       62739 non-null  object        
 8   class                          62739 non-null  int64         
 9   gender                         62739 non-null  object        
 10  anchor_age                     62739 non-null  int64         
 11  anchor_year    

In [ ]:
merged_micro[merged_micro['subject_id']==10191244]

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,organisms_found,first_culture_time,spec_types
1231,10191244,27922516,NaN,F419,"Anxiety disorder, unspecified",6,common_diseases,True,4,F,22,2165,2017 - 2019,NaT,2165-10-23 23:35:00,2165-10-24 11:17:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2165-10-23 21:31:00,2165-10-24 11:17:00,0,1,0,0,"F419,L732",0,0.487,Fall,-2.067,10,1,0,0,0,0,NaN,NaT,0,2,1,Emergency Department Observation,Emergency Department;Emergency Department Obse...,Emergency Department Observation,0,1,0,0,0,0,7,0,1,3,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
filtered_microbiologyevents[filtered_microbiologyevents['subject_id']==10191244]

,microevent_id,subject_id,hadm_id,micro_specimen_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,storedate,storetime,test_itemid,test_name,org_itemid,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments,spec_type_desc_str
5066,NaN,10191244,27922516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan


#### 8. Pharmacy Table

In [ ]:
# Filter pharmacy to include only rows with subject_id and hadm_id in main merged dataframe till now
filtered_pharmacy = pharmacy.merge(merged_micro[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')

#Define route classification function
def classify_route_group(route):
    if pd.isna(route):
        return 'Others'
    route_upper = route.upper()

    if any(x in route_upper for x in ['PO', 'ORAL', 'NG', 'PO/NG']):
        return 'Oral'
    elif 'IV' in route_upper:
        return 'IV'
    elif any(x in route_upper for x in ['EYE', 'OU']):
        return 'Eyes'
    elif any(x in route_upper for x in ['TP', 'TD', 'SC', 'SUBCUT', 'ID', 'PERIPHNERVE', 'VG']):
        return 'Skin'
    else:
        return 'Others'

#Create new column with route group
filtered_pharmacy['route_group'] = filtered_pharmacy['route'].apply(classify_route_group)

#1. Count medications by route group per subject_id and hadm_id
route_group_counts = (
    filtered_pharmacy
    .groupby(['subject_id', 'hadm_id', 'route_group'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

#Rename columns to clarify they are route group counts
route_group_counts.columns = ['subject_id', 'hadm_id'] + [f'route_{col.lower()}' for col in route_group_counts.columns[2:]]

#Merging this into already merged main dataframe
merged_pharmacy = merged_micro.merge(route_group_counts, on=['subject_id', 'hadm_id'], how='left')

In [ ]:
#2. Total number of medications
med_counts = filtered_pharmacy.groupby(['subject_id', 'hadm_id'])['medication'].count().reset_index(name='total_medications')

#3. Number of unique medications
unique_meds = filtered_pharmacy.groupby(['subject_id', 'hadm_id'])['medication'].nunique().reset_index(name='unique_medications')

# Number of PRN medications (get from frequency column)
prn_meds = (filtered_pharmacy[filtered_pharmacy['frequency'].str.contains('PRN', case=False, na=False)].groupby(['subject_id', 'hadm_id'])['medication']
            .count().reset_index(name='prn_med_count'))

#4. First medication administration time
# Ensure 'starttime' is datetime, coercing errors to NaT
filtered_pharmacy['starttime'] = pd.to_datetime(filtered_pharmacy['starttime'], errors='coerce')
first_med_time = (filtered_pharmacy.groupby(['subject_id', 'hadm_id'])['starttime'].min().reset_index(name='first_med_time'))

#5. Duration stats (requires non-null starttime & stoptime)
filtered_pharmacy['stoptime'] = pd.to_datetime(filtered_pharmacy['stoptime'], errors='coerce') # Also convert stoptime
# Calculate med_duration only for rows where both starttime and stoptime are valid datetimes
filtered_pharmacy['med_duration'] = pd.to_datetime(filtered_pharmacy['stoptime']) - pd.to_datetime(filtered_pharmacy['starttime'])

duration_stats = (filtered_pharmacy.groupby(['subject_id', 'hadm_id'])['med_duration']
                  .agg(total_med_duration='sum', avg_med_duration='mean', max_med_duration='max').reset_index())

#6. List of medications (semicolon-separated)
# Filter out non-string types (like NaNs), convert remaining to string, then sort and join
med_list = (filtered_pharmacy.groupby(['subject_id', 'hadm_id'])['medication'].unique().apply(lambda meds: '; '.join(sorted(set(str(m) for m in meds if isinstance(m, str)))))
    .reset_index(name='medications_given'))

# Merge everything into one dataframe
merged_pharmacy_temp = med_counts \
    .merge(unique_meds, on=['subject_id', 'hadm_id'], how='outer') \
    .merge(med_list, on=['subject_id', 'hadm_id'], how='outer') \
    .merge(prn_meds, on=['subject_id', 'hadm_id'], how='outer') \
    .merge(first_med_time, on=['subject_id', 'hadm_id'], how='outer') \
    .merge(duration_stats, on=['subject_id', 'hadm_id'], how='outer')

# Merge merged_pharmacy_temp into merged_pharmacy on subject_id and hadm_id
merged_pharmacy = merged_pharmacy.merge(merged_pharmacy_temp, on=['subject_id', 'hadm_id'], how='left')

In [ ]:
merged_pharmacy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 94 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   subject_id                     62739 non-null  Int64          
 1   hadm_id                        62739 non-null  Int64          
 2   procedures_applied             32543 non-null  object         
 3   icd_code                       62739 non-null  object         
 4   long_title                     62739 non-null  object         
 5   seq_num                        62739 non-null  int64          
 6   Classification                 62739 non-null  object         
 7   included                       62739 non-null  object         
 8   class                          62739 non-null  int64          
 9   gender                         62739 non-null  object         
 10  anchor_age                     62739 non-null  int64          
 11  an

#### 9. Prescriptions Table

In [ ]:
#Filter prescriptions to include only rows with subject_id and hadm_id in main merged dataframe till now
filtered_prescriptions = prescriptions.merge(merged_pharmacy[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')

#1. Getting counts of total prescriptions, unique prescriptions, list of drugs prescribed (not necessary patient took it)
prescription_summary = (
    filtered_prescriptions
    .groupby(['subject_id', 'hadm_id'])
    .agg(num_prescriptions_drugs=('drug', 'count'),
        num_unique_prescriptions_drugs=('drug', pd.Series.nunique),
        # Handle potential non-string values (like NaN) by converting to string before lowercasing
        drug_list=('drug', lambda x: ', '.join(sorted(set(str(m).lower() for m in x if pd.notna(m)))) )
    )
    .reset_index()
)

#2. Apply classification function to create a new column
filtered_prescriptions['route_group'] = filtered_prescriptions['route'].apply(classify_route_group)
# Count number of prescriptions per route group
route_counts = (filtered_prescriptions.groupby(['subject_id', 'hadm_id', 'route_group']).size().unstack(fill_value=0).reset_index().rename_axis(None, axis=1))
# Automatically rename route columns to include '_prescribed_route'
route_counts_renamed = route_counts.rename(columns={
    col: f"{col.lower()}_prescribed_route"
    for col in route_counts.columns
    if col not in ['subject_id', 'hadm_id']
})

#3.Duration & Dosage values
filtered_prescriptions['dose_val_rx'] = pd.to_numeric(filtered_prescriptions['dose_val_rx'], errors='coerce')
filtered_prescriptions['starttime'] = pd.to_datetime(filtered_prescriptions['starttime'])
filtered_prescriptions['stoptime'] = pd.to_datetime(filtered_prescriptions['stoptime'])
# Calculate duration in days
filtered_prescriptions['duration_days'] = (filtered_prescriptions['stoptime'] - filtered_prescriptions['starttime']).dt.total_seconds() / (60 * 60 * 24)
dose_duration_stats = (filtered_prescriptions.groupby(['subject_id', 'hadm_id']).agg(
        avg_dose_val=('dose_val_rx', 'mean'),
        max_dose_val=('dose_val_rx', 'max'),
        avg_duration_days=('duration_days', 'mean'),
        max_duration_days=('duration_days', 'max')).reset_index())

#Merging all the different values extracted into the main dataframe
merged_prescriptions = (
    merged_pharmacy
    .merge(prescription_summary, on=['subject_id', 'hadm_id'], how='left')
    .merge(route_counts_renamed, on=['subject_id', 'hadm_id'], how='left')
    .merge(dose_duration_stats, on=['subject_id', 'hadm_id'], how='left')
)

In [ ]:
merged_prescriptions.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 106 columns):
 #    Column                          Non-Null Count  Dtype          
---   ------                          --------------  -----          
 0    subject_id                      62739 non-null  Int64          
 1    hadm_id                         62739 non-null  Int64          
 2    procedures_applied              32543 non-null  object         
 3    icd_code                        62739 non-null  object         
 4    long_title                      62739 non-null  object         
 5    seq_num                         62739 non-null  int64          
 6    Classification                  62739 non-null  object         
 7    included                        62739 non-null  object         
 8    class                           62739 non-null  int64          
 9    gender                          62739 non-null  object         
 10   anchor_age                      62739 non-nu

In [ ]:
#Extracting more features from existing Pharmacy and Prescriptions data.
#Understand the difference between Pharmacy and Prescriptions table, Pharmacy is more accurate.

from collections import Counter

# Function to extract drugs from a column like 'aspirin; metformin; insulin'
def extract_all_drugs(drug_series):
    all_drugs = []
    for drug_list in drug_series.dropna():
        drugs = [d.strip().lower() for d in drug_list.split(';') if d.strip()]
        all_drugs.extend(drugs)
    return all_drugs

# Check if 'medications_given' column exists (Pharmacy)
if 'medications_given' in merged_prescriptions.columns:
    print("Processing merged_prescriptions...")
    drugs = extract_all_drugs(merged_prescriptions['medications_given'])
    drug_counter = Counter(drugs)

    # Convert to DataFrame
    drug_freq_df_given = pd.DataFrame(drug_counter.items(), columns=['drug_name', 'frequency'])
    drug_freq_df_given = drug_freq_df_given.sort_values(by='frequency', ascending=False).reset_index(drop=True)
else:
    print("'medications_given' column not found in merged_prescriptions.")

# Check if 'drug_list' column exists (Prescriptions)
if 'drug_list' in merged_prescriptions.columns:
    print("Processing merged_prescriptions...")
    drugs = extract_all_drugs(merged_prescriptions['drug_list'])
    drug_counter = Counter(drugs)

    # Convert to DataFrame
    drug_freq_df = pd.DataFrame(drug_counter.items(), columns=['drug_name', 'frequency'])
    drug_freq_df = drug_freq_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)
else:
    print("'drug_list' column not found in merged_prescriptions.")


Processing merged_prescriptions...
Processing merged_prescriptions...


In [ ]:
drug_freq_df_given.head(10)

,drug_name,frequency
0,sodium chloride 0.9% flush,51649
1,acetaminophen,44595
2,heparin,38641
3,docusate sodium,31636
4,senna,30994
5,influenza vaccine quadrivalent,26068
6,ondansetron,23459
7,oxycodone (immediate release),22505
8,aspirin,19781
9,dextrose 50%,19776


In [ ]:
drug_freq_df.head(10)

,drug_name,frequency
0,acetaminophen,13
1,"acetaminophen, pregabalin",12
2,"heparin, sodium chloride 0.9% flush",10
3,heparin,9
4,"acetaminophen, heparin, sodium chloride 0.9% ...",8
5,"acetaminophen, bisacodyl, calcium carbonate, d...",7
6,"acetaminophen, heparin, influenza vaccine quad...",6
7,"acetaminophen, heparin",5
8,influenza vaccine quadrivalent,4
9,"heparin, influenza vaccine quadrivalent, sodiu...",4


Both (pharmacy and prescriptions) have more or less similar drugs so we have created a mapping of common drug classes using FDA data, of the top 100 drugs given to the patients.
https://www.fda.gov/drugs/investigational-new-drug-ind-application/general-drug-categories

In [ ]:
# Define the drug category mapping
drug_category_keywords = {
    'has_insulin': ['insulin', 'glucagon'],
    'has_heparin': ['heparin', 'enoxaparin', 'warfarin'],
    'has_aspirin': ['aspirin'],
    'has_ibuprofen': ['ibuprofen'],
    'has_fluids': ['sodium chloride', 'dextrose', 'lactated ringers', 'ns', 'sterile water', 'd5 1/2ns'],
    'has_antibiotics': ['vancomycin', 'ceftriaxone', 'cefepime', 'metronidazole', 'cefazolin', 'azithromycin',
                        'ciprofloxacin', 'levofloxacin', 'piperacillin-tazobactam'],
    'has_analgesics': ['acetaminophen', 'oxycodone', 'tramadol', 'hydromorphone', 'morphine', 'fentanyl', 'ibuprofen'],
    'has_antifungal': ['fluconazole', 'ketoconazole', 'clotrimazole', 'terbinafine', 'itraconazole'],
    'has_steroid': ['prednisone', 'hydrocortisone', 'methylprednisolone', 'dexamethasone'],
    'has_topical': ['cream', 'ointment', 'gel', 'lotion', 'patch', 'neb soln'],
    'has_immunosuppressant': ['methotrexate', 'cyclosporine', 'azathioprine', 'mycophenolate', 'tacrolimus'],
    'has_antihistamine': ['diphenhydramine', 'loratadine', 'cetirizine', 'fexofenadine'],
    'has_moisturizer': ['emollient', 'moisturizer', 'aquaphor', 'vaseline', 'sarna lotion'],
    'has_antiviral': ['acyclovir', 'valacyclovir', 'famciclovir'],
    'has_antidepressant': ['fluoxetine', 'sertraline', 'trazodone', 'amitriptyline'],
    'has_pain_management': ['gabapentin', 'pregabalin', 'tramadol', 'fentanyl', 'oxycodone'],
    'has_diuretics': ['furosemide', 'hydrochlorothiazide', 'spironolactone'],
    'has_vaccine': ['influenza vaccine', 'pneumococcal vaccine', 'hepatitis vaccine'],
    'has_ace_inhibitor': ['lisinopril', 'enalapril', 'ramipril'],
    'has_statins': ['atorvastatin', 'simvastatin', 'pravastatin']
}

# Function to add feature columns to a dataset
def add_drug_features(df):
    # Ensure drug list is lowercase for matching and handle NaN values
    # Check if the entry is not NaN and is iterable (like a list or string) before processing
    df['drug_list_clean'] = df['medications_given'].apply(
        lambda lst: [str(d).strip().lower() for d in lst.split(';')] if pd.notna(lst) else []
    )

    # Add one column per drug category
    for feature, keywords in drug_category_keywords.items():
        df[feature] = df['drug_list_clean'].apply(
            lambda drugs: int(any(any(keyword in drug for keyword in keywords) for drug in drugs))
        )

    return df.drop(columns=['drug_list_clean'])

# Apply to merged_prescriptions
extracted_prescriptions = add_drug_features(merged_prescriptions)

In [ ]:
# Ensure datetime conversion for first_med_time
extracted_prescriptions['first_med_time'] = pd.to_datetime(extracted_prescriptions['first_med_time'], errors='coerce')

# 1. Time to first medication in hours
extracted_prescriptions['time_to_first_med_hrs'] = (
    (extracted_prescriptions['first_med_time'] - extracted_prescriptions['admittime'])
    .dt.total_seconds() / 3600
)

# 2. Difference in medication counts
extracted_prescriptions['diff_in_medicines'] = (
    extracted_prescriptions['num_unique_prescriptions_drugs'] - extracted_prescriptions['unique_medications']
).abs()

# 3. Polypharmacy flag (1 if patient has >5 unique medications)
extracted_prescriptions['polypharmacy'] = (extracted_prescriptions['unique_medications'] > 5).astype(int)

In [ ]:
extracted_prescriptions.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 129 columns):
 #    Column                          Non-Null Count  Dtype          
---   ------                          --------------  -----          
 0    subject_id                      62739 non-null  Int64          
 1    hadm_id                         62739 non-null  Int64          
 2    procedures_applied              32543 non-null  object         
 3    icd_code                        62739 non-null  object         
 4    long_title                      62739 non-null  object         
 5    seq_num                         62739 non-null  int64          
 6    Classification                  62739 non-null  object         
 7    included                        62739 non-null  object         
 8    class                           62739 non-null  int64          
 9    gender                          62739 non-null  object         
 10   anchor_age                      62739 non-nu

In [ ]:
extracted_prescriptions.tail()

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,organisms_found,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,total_med_duration,avg_med_duration,max_med_duration,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy
62734,19999043,24799384,"Extraction of Products of Conception, Retained...",F419,"Anxiety disorder, unspecified",4,common_diseases,True,4,F,35,2164,2014 - 2016,NaT,2164-12-18 22:42:00,2164-12-24 12:45:00,NaN,OBSERVATION ADMIT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,2164-12-18 11:11:00,2164-12-18 17:37:00,0,1,0,0,F419,0,5.585,Winter,-11.517,12,1,0,5,5,2,32.800,2164-10-28,3,3,1,PACU,Emergency Department;PACU;Med/Surg/GYN;Med/Sur...,Med/Surg/GYN,0,5,4,0,0,0,21,0,6,14,68,0,9,0,4,2,0,0,0,0,0,0,0,1,2,0,2,0,0,NaN,2164-12-22 01:50:00,IMMUNOLOGY; URINE,0,20,28,0,0,46,22,Acetaminophen; Acetaminophen IV; CefTRIAXone; ...,26.000,2164-12-18 00:00:00,77 days 02:00:00,1 days 14:32:30,4 days 14:00:00,51,25,"0.9% sodium chloride (mini bag plus), 5% dextr...",0,23,28,0,0,182.972,1000.000,1.560,4.583,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-22.700,3,1
62735,19999464,23033573,"Excision of Sigmoid Colon, Via Natural or Arti...",Q453,Other congenital malformations of pancreas and...,2,Uncategorised,True,5,F,35,2160,2008 - 2010,NaT,2171-08-01 18:34:00,2171-08-04 16:20:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,2171-08-01 15:12:00,2171-08-01 19:31:00,0,1,0,0,Q453,0,2.907,Summer,-3.367,8,0,0,2,2,0,24.900,2171-08-01,0,2,1,Med/Surg,Emergency Department;Med/Surg;nan,Med/Surg,0,6,0,0,1,0,8,0,1,19,27,0,2,0,0,0,0,0,0,0,6,0,0,0,6,1,5,0,0,CANCELLED,2171-08-02 11:51:00,STOOL,0,6,11,0,1,18,11,Acetaminophen; Acetaminophen-Caff-Butalbital; ...,2.000,2171-08-01 21:00:00,-54 days +01:00:00,-3 days +00:03:20,3 days 00:00:00,19,12,"0.9% sodium chloride, acetaminophen, acetamino...",0,7,11,0,1,427.118,5000.000,-2.789,3.000,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,2.433,1,1
62736,19999466,21397174,NaN,I10,Essential (primary) hypertension,2,Uncategorised,True,5,M,33,2116,2017 - 2019,NaT,2116-08-30 15:21:00,2116-08-31 21:49:00,NaN,DIRECT OBSERVATION,CLINIC REFERRAL,NaN,Medicare,ENGLISH,NaN,OTHER,2116-08-30 12:11:00,2116-08-31 21:49:00,0,1,0,0,I10,0,1.269,Summer,-3.167,8,0,1,0,0,0,NaN,NaT,0,2,1,Em

#### 10. Labevents Table

In [ ]:
%%time
#Combining labevents and labitems
combinedlabs = pd.merge(labevents, labitems, on='itemid', how='left')

CPU times: user 31.9 s, sys: 1min 10s, total: 1min 42s
Wall time: 2min 11s


In [ ]:
print(extracted_prescriptions[['subject_id', 'hadm_id']].dtypes)
print(combinedlabs[['subject_id', 'hadm_id']].dtypes)

subject_id    Int64
hadm_id       Int64
dtype: object
subject_id      int64
hadm_id       float64
dtype: object


In [ ]:
%%time
# Align data types across both DataFrames
combinedlabs['subject_id'] = combinedlabs['subject_id'].astype('Int64')
combinedlabs['hadm_id'] = combinedlabs['hadm_id'].astype('Int64')

CPU times: user 31.9 s, sys: 20.4 s, total: 52.3 s
Wall time: 1min


In [ ]:
%%time
# Filter labevents to include only rows with subject_id and hadm_id in main merged dataframe till now
filtered_labevents = combinedlabs.merge(extracted_prescriptions[['subject_id', 'hadm_id']].drop_duplicates(),on=['subject_id', 'hadm_id'],how='right')

CPU times: user 42.9 s, sys: 33.4 s, total: 1min 16s
Wall time: 1min 31s


In [ ]:
#Checking a record
filtered_labevents[filtered_labevents['subject_id'] == 10136839]

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category
179539,1546436.000,10136839,29401107,22176238.000,51221.000,2175-05-07 03:10:00,2175-05-07 04:19:00,29.4,29.400,%,40.000,51.000,abnormal,ROUTINE,NaN,Hematocrit,Blood,Hematology
179540,1546437.000,10136839,29401107,22176238.000,51222.000,2175-05-07 03:10:00,2175-05-07 04:19:00,9.9,9.900,g/dL,13.700,17.500,abnormal,ROUTINE,NaN,Hemoglobin,Blood,Hematology
179541,1546438.000,10136839,29401107,22176238.000,51248.000,2175-05-07 03:10:00,2175-05-07 04:19:00,31.4,31.400,pg,26.000,32.000,NaN,ROUTINE,NaN,MCH,Blood,Hematology
179542,1546439.000,10136839,29401107,22176238.000,51249.000,2175-05-07 03:10:00,2175-05-07 04:19:00,33.7,33.700,g/dL,32.000,37.000,NaN,ROUTINE,NaN,MCHC,Blood,Hematology
179543,1546440.000,10136839,29401107,22176238.000,51250.000,2175-05-07 03:10:00,2175-05-07 04:19:00,93,93.000,fL,82.000,98.000,NaN,ROUTINE,NaN,MCV,Blood,Hematology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181772,1548669.000,10136839,29401107,86942233.000,52069.000,2175-06-05 00:00:00,2175-06-05 03:13:00,0.00,0.000,K/uL,0.010,0.080,abnormal,ROUTINE,NaN,Absolute Basophil Count,Blood,Hematology
181773,1548670.000,10136839,29401107,86942233.000,52073.000,2175-06-05 00:00:00,2175-06-05 03:13:00,0.00,0.000,K/uL,0.040,0.540,abnormal,ROUTINE,NaN,Absolute Eosinophil Count,Blood,Hematology
181774,1548671.000,10136839,29401107,86942233.000,52074.000,2175-06-05 00:00:00,2175-06-05 03:13:00,0.08,0.080,K/uL,0.200,0.800,abnormal,ROUTINE,NaN,Absolute Monocyte Count,Blood,Hematology
181775,1548672.000,10136839,29401107,86942233.000,52075.000,2175-06-05 00:00:00,2175-06-05 03:13:00,1.09,1.090,K/uL,1.600,6.100,abnormal,ROUTINE,NaN,Absolute Neutrophil Count,Blood,Hematology


In [ ]:
%%time
# Make sure charttime is datetime type
filtered_labevents['charttime'] = pd.to_datetime(filtered_labevents['charttime'])

# Define a helper to check if flag contains 'abnormal' (case-insensitive)
def is_abnormal_flag(flag):
    if pd.isna(flag):
        return False
    return 'abnormal' in flag.lower()

filtered_labevents['is_abnormal'] = filtered_labevents['flag'].apply(is_abnormal_flag)

# Aggregate lab stats per admission
lab_agg = filtered_labevents.groupby(['subject_id', 'hadm_id']).apply(lambda df: pd.Series({
    'total_lab_tests': len(df),
    'unique_lab_types': df['label'].nunique(),
    'abnormal_lab_count': df['is_abnormal'].sum(),
    'first_lab_time': df['charttime'].min(),
    'first_abnormal_lab_time': df.loc[df['is_abnormal'], 'charttime'].min() if df['is_abnormal'].any() else pd.NaT
})).reset_index()

# Frequency of each lab test per admission in wide format
lab_freq = (
    filtered_labevents
    .groupby(['subject_id', 'hadm_id', 'label'])
    .size()
    .unstack(fill_value=0)
    .reset_index())

# Filter out rows without numeric values
numeric_labs = filtered_labevents[filtered_labevents['valuenum'].notna()]

# Sort by charttime so we can easily extract first/last
numeric_labs_sorted = numeric_labs.sort_values(['subject_id', 'hadm_id', 'label', 'charttime'])

# Group by subject_id, hadm_id, label and extract relevant stats
lab_agg1 = numeric_labs_sorted.groupby(['subject_id', 'hadm_id', 'label']).agg(
    first_value=('valuenum', 'first'),
    max_value=('valuenum', 'max'),
    min_value=('valuenum', 'min'),
    last_value=('valuenum', 'last')
).reset_index()

# Pivot so each lab test becomes a column (wide format)
lab_values_pivot = lab_agg1.pivot(index=['subject_id', 'hadm_id'], columns='label')

# Flatten multiindex column names
lab_values_pivot.columns = [
    f"{stat}_{label}".lower().replace(" ", "_")
    for stat, label in lab_values_pivot.columns
]

lab_values_pivot = lab_values_pivot.reset_index()

CPU times: user 1min 45s, sys: 13.3 s, total: 1min 58s
Wall time: 2min 1s


In [ ]:
%%time
#Merging all the extracted features into main dataframe
merged_labs = (extracted_prescriptions
    .merge(lab_agg, on=['subject_id', 'hadm_id'], how='left')
    .merge(lab_values_pivot, on=['subject_id', 'hadm_id'], how='left')
    .merge(lab_freq, on=['subject_id', 'hadm_id'], how='left'))

CPU times: user 3.39 s, sys: 3.73 s, total: 7.12 s
Wall time: 8.31 s


In [ ]:
merged_labs.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 2386 columns):
 #     Column                                                  Non-Null Count  Dtype          
---    ------                                                  --------------  -----          
 0     subject_id                                              62739 non-null  Int64          
 1     hadm_id                                                 62739 non-null  Int64          
 2     procedures_applied                                      32543 non-null  object         
 3     icd_code                                                62739 non-null  object         
 4     long_title                                              62739 non-null  object         
 5     seq_num                                                 62739 non-null  int64          
 6     Classification                                          62739 non-null  object         
 7     included                   

In [ ]:
merged_labs.tail()

,subject_id,hadm_id,procedures_applied,icd_code,long_title,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,is_dead,los,admission_season,time_to_emergency_hrs,admit_month,is_night_admit,is_weekend_admit,total_no_of_procedures,no_of_unique_procedures,count_icu_critical_procedures,bmi,bmi_chartdate,no_of_transfers,no_of_distinct_careunits,no_of_ed_transfers,first_careunit,careunit_transfer_sequence,last_careunit,no_of_icu_stays,count_adt_orders,count_blood_bank,count_cardiology,count_consults,count_critical_care,count_general_care,count_hemodialysis,count_iv_therapy,count_lab,count_medications,count_neurology,count_nutrition,count_ob,count_radiology,count_respiratory,count_tpn,count_abscess,count_blood_culture,count_sputum,count_stool,count_swab,count_tissue,count_urine,total_microbiologyevents,positive_cultures,negative_cultures,susceptible_tests,resistant_tests,organisms_found,first_culture_time,spec_types,route_eyes,route_iv,route_oral,route_others,route_skin,total_medications,unique_medications,medications_given,prn_med_count,first_med_time,total_med_duration,avg_med_duration,max_med_duration,num_prescriptions_drugs,num_unique_prescriptions_drugs,drug_list,eyes_prescribed_route,iv_prescribed_route,oral_prescribed_route,others_prescribed_route,skin_prescribed_route,avg_dose_val,max_dose_val,avg_duration_days,max_duration_days,has_insulin,has_heparin,has_aspirin,has_ibuprofen,has_fluids,has_antibiotics,has_analgesics,has_antifungal,has_steroid,has_topical,has_immunosuppressant,has_antihistamine,has_moisturizer,has_antiviral,has_antidepressant,has_pain_management,has_diuretics,has_vaccine,has_ace_inhibitor,has_statins,time_to_first_med_hrs,diff_in_medicines,polypharmacy,total_lab_tests,unique_lab_types,abnormal_lab_count,first_lab_time,first_abnormal_lab_time,first_value_%_hemoglobin_a1c,first_value_24_hr_calcium,first_value_24_hr_creatinine,first_value_24_hr_protein,first_value_25-oh_vitamin_d,first_value_arch-1,first_value_absolute_basophil_count,first_value_absolute_cd3_count,first_value_absolute_cd4_count,first_value_absolute_cd8_count,first_value_absolute_eosinophil_count,first_value_absolute_lymphocyte_count,first_value_absolute_monocyte_count,first_value_absolute_neutrophil_count,first_value_acetaminophen,first_value_alanine_aminotransferase_(alt),first_value_albumin,"first_value_albumin,_ascites","first_value_albumin,_body_fluid","first_value_albumin,_pleural","first_value_albumin,_urine","first_value_albumin/creatinine,_urine",first_value_alkaline_phosphatase,first_value_alpha-fetoprotein,first_value_alveolar-arterial_gradient,first_value_amikacin,first_value_ammonia,first_value_amylase,"first_value_amylase,_ascites","first_value_amylase,_body_fluid","first_value_amylase,_pleural","first_value_amylase,_urine","first_value_amylase/creatinine_ratio,_urine",first_value_anion_gap,first_value_anisocytosis,first_value_anti-dgp_(iga/igg),first_value_anti-thyroglobulin_antibodies,first_value_antithrombin,first_value_asparate_aminotransferase_(ast),first_value_atypical_lymphocytes,first_value_bacteria,first_value_bands,first_value_base_excess,first_value_basophilic_stippling,first_value_basophils,first_value_beta_hydroxybutyrate,first_value_beta-2_microglobulin,first_value_bicarbonate,"first_value_bicarbonate,_stool","first_value_bicarbonate,_urine","first_value_bilirubin,_direct","first_value_bilirubin,_indirect","first_value_bilirubin,_total","first_value_bilirubin,_total,_ascites","first_value_bilirubin,_total,_body_fluid","first_value_bilirubin,_total,_csf","first_value_bilirubin,_total,_pleural",first_value_blasts,first_value_blood_parasite_smear,first_value_broad_casts,first_value_c-reactive_protein,first_value_c3,first_value_c4,first_value_ca_19-9,first_valu

In [ ]:
%%time
#Applying conditional fixes because of table merge
def apply_conditional_fixes(df):
    # Condition: If first_lab_time is NaT
    mask_lab_nat = df['first_lab_time'].isna()
    df.loc[mask_lab_nat, 'total_lab_tests'] = 0

    return df

merged_labs = apply_conditional_fixes(merged_labs)

CPU times: user 1.15 s, sys: 1.47 s, total: 2.62 s
Wall time: 3.02 s


In [ ]:
%%time
#Extracting phenotype features
# === STEP 1: Define phenotype groupings ===
phenotype_groups = {
    'Renal': ['first_value_creatinine', 'first_value_urea_nitrogen'],
    'Electrolyte_Fluid': ['first_value_sodium', 'first_value_potassium', 'first_value_chloride',
                          'first_value_calcium,_total', 'first_value_magnesium', 'first_value_phosphate'],
    'Hematologic': ['first_value_hemoglobin', 'first_value_hematocrit', 'first_value_red_blood_cells',
                    'first_value_platelet_count', 'first_value_white_blood_cells'],
    'Metabolic_AcidBase': ['first_value_anion_gap', 'first_value_bicarbonate'],
    'RBC_Indices': ['first_value_mcv', 'first_value_mch', 'first_value_mchc', 'first_value_rdw', 'first_value_rdw-sd'],
    'Glucose_Metabolic': ['first_value_glucose']
}

# === STEP 2: Define clinical reference ranges ===
reference_ranges = {
    'first_value_creatinine': (0.5, 1.3),
    'first_value_urea_nitrogen': (7, 20),
    'first_value_sodium': (135, 145),
    'first_value_potassium': (3.5, 5.0),
    'first_value_chloride': (98, 107),
    'first_value_calcium,_total': (8.5, 10.5),
    'first_value_magnesium': (1.5, 2.5),
    'first_value_phosphate': (2.5, 4.5),
    'first_value_hemoglobin': (12.0, 16.0),
    'first_value_hematocrit': (36.0, 46.0),
    'first_value_red_blood_cells': (4.0, 5.4),
    'first_value_platelet_count': (150, 450),
    'first_value_white_blood_cells': (4.0, 11.0),
    'first_value_anion_gap': (8, 16),
    'first_value_bicarbonate': (22, 29),
    'first_value_mcv': (80, 100),
    'first_value_mch': (27, 33),
    'first_value_mchc': (32, 36),
    'first_value_rdw': (11.5, 14.5),
    'first_value_rdw-sd': (37, 54),
    'first_value_glucose': (70, 110)
}

# === STEP 3: Function to check abnormality ===
def is_abnormal(val, low, high):
    if pd.isnull(val): return False
    return val < low or val > high

# === STEP 4: Function to compute phenotype flags for a row ===
def get_phenotype_flags(row):
    flags = []
    for phenotype, labs in phenotype_groups.items():
        if any(
            is_abnormal(row.get(lab), *reference_ranges.get(lab, (-np.inf, np.inf)))
            for lab in labs if lab in row
        ):
            flags.append(f'has_abnormal_{phenotype.lower()}')
    return ', '.join(flags) if flags else 'normal'

# === STEP 5: Apply to merged_labs ===
merged_labs['phenotype_flags'] = merged_labs.apply(get_phenotype_flags, axis=1)


CPU times: user 1min 20s, sys: 16.4 s, total: 1min 36s
Wall time: 1min 39s


In [ ]:
merged_labs.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62739 entries, 0 to 62738
Data columns (total 2387 columns):
 #     Column                                                  Non-Null Count  Dtype          
---    ------                                                  --------------  -----          
 0     subject_id                                              62739 non-null  Int64          
 1     hadm_id                                                 62739 non-null  Int64          
 2     procedures_applied                                      32543 non-null  object         
 3     icd_code                                                62739 non-null  object         
 4     long_title                                              62739 non-null  object         
 5     seq_num                                                 62739 non-null  int64          
 6     Classification                                          62739 non-null  object         
 7     included                   

In [ ]:
%%time
# write the cleaned data frame to a new Excel file
merged_labs.to_excel("merged_labs.xlsx", index=False, header = True)

CPU times: user 26min 37s, sys: 31.9 s, total: 27min 9s
Wall time: 1h 55min 51s


In [ ]:
%%time
# Write the cleaned DataFrame to a CSV file
merged_labs.to_csv("merged_labs.csv", index=False, header = True)

CPU times: user 1min 18s, sys: 2.84 s, total: 1min 21s
Wall time: 1min 22s
